In [1]:
!pip install streamlit pyngrok opencv-python-headless numpy ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 841.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.9/871.9 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [2]:
%%writefile object_tracking.py
import streamlit as st
import cv2
import numpy as np
import tempfile
import os
from ultralytics import YOLO

#load model
model = YOLO('yolov8n.pt')

#define function to process frames
def process_frame(frame):
  frame_resized = cv2.resize(frame, (640,480))

  #perform object detection
  results = model(frame_resized, stream=True)

  #draw bounding boxes on the frame
  for r in results:
    if len(r.boxes) >0:
      x1, y1, x2, y2 = box.xyxy[0].int().tolist()
      conf = box.conf[0].item() #scores
      cls = int(box.cls[0].item()) #class
      label = model.names[cls]
      color = (0,255,0)
      frame_resized = cv2.rectangle(frame_resized, (x1,y1), (x2,y2), color, 2)
      frame_resized = cv2.putText(frame_resized, f'{label}: {conf:.2f}', (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    frame_resized = cv2.resize(frame_resized, (frame.shape[1], frame.shape[0]))
  return frame_resized

def main():
  st.title('Object Tracking')
  video_file = st.file_uploader('Upload a video', type=['mp4', 'avi', 'mov'])

  if video_file:
    with tempfile.NamedTemporaryFile(delete = False) as tmp_file:
      tmp_file.write(video_file.read())
      temp_file_path = tmp_file.name

    cap = cv2.VideoCapture(temp_file_path)

    stframe = st.empty()

    while cap.isOpened():
      ret, frame = cap.read()
      if not ret:
        break

      frame = process_frame(frame)
      rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      stframe.image(rgb_frame, channels = 'RGB', use_column_width= True)

    cap.release()
    os.remove(temp_file_path)

if __name__ == '__main__':
  main()

Writing object_tracking.py


In [3]:
#setup ngrok with your authonken
!ngrok authtoken 2JQ7xQIMC37Y473rZ6ZbapzlPMB_4cZK6X28eFyAPsEfKCwD8

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [4]:
from pyngrok import ngrok
!streamlit run object_tracking.py &>/dev/null&

In [5]:
!pgrep -f streamlit

4210


In [6]:
public_url = ngrok.connect(8501)
public_url

<NgrokTunnel: "https://e044-34-83-237-202.ngrok-free.app" -> "http://localhost:8501">